# Exploring Ebay Car Sales Data
Author: Jimmy Nguyen

## Introduction

In this project, I look at car sales data from *eBay Kleinanzeigen*, a classifieds section of the German eBay website.

Data dictionary for the raw data provided:

*  `dateCrawled` - When this ad was first crawled. All field-values are taken from this date.
*  `name` - Name of the car.
*  `seller` - Whether the seller is private or a dealer.
*  `offerType` - The type of listing
*  `price` - The price on the ad to sell the car.
*  `abtest` - Whether the listing is included in an A/B test.
*  `vehicleType` - The vehicle Type.
*  `yearOfRegistration` - The year in which the car was first registered.
*  `gearbox` - The transmission type.
*  `powerPS` - The power of the car in PS.
*  `model` - The car model name.
*  `kilometer` - How many kilometers the car has driven.
*  `monthOfRegistration` - The month in which the car was first registered.
*  `fuelType` - What type of fuel the car uses.
*  `brand` - The brand of the car.
*  `notRepairedDamage` - If the car has a damage which is not yet repaired.
*  `dateCreated` - The date on which the eBay listing was created.
*  `nrOfPictures` - The number of pictures in the ad.
*  `postalCode` - The postal code for the location of the vehicle.
*  `lastSeenOnline` - When the crawler saw this ad last online.

In [172]:
# Import data analysis libraries
import numpy as np
import pandas as pd

In [173]:
# Read in eBay car sales dataset
autos = pd.read_csv("autos.csv", encoding = "Latin-1")

## Examining The Data
This dataset has 50,000 rows and 20 features. Each row is a car sale containing information about the seller and the car sold.

In [174]:
# Print basic information about the dataset
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## Data Cleaning

### Cleaning up column names
Let's convert the column names to Python's preferred `snake_case` and rename the columns to facilitate working with the data.

We'll accomplish this by manually renaming some columns, and then using `regex` matching to insert underscores between camelCase words, and then finally making all column names lower case using `Series.str.lower()`.

In [175]:
# Convert column names from camelCase to snake_case
autos.columns = (autos.columns
         .str.replace('yearOfRegistration', 'registration_year')
         .str.replace('monthOfRegistration', 'registration_month')
         .str.replace('notRepairedDamage', 'unrepaired_damage')
         .str.replace('dateCreated', 'ad_created')
         .str.replace(r'([A-Z][a-z]+)', r'_\1') # insert _ between words
         .str.lower()
    )
list(autos)

['date_crawled',
 'name',
 'seller',
 'offer_type',
 'price',
 'abtest',
 'vehicle_type',
 'registration_year',
 'gearbox',
 'powerps',
 'model',
 'odometer',
 'registration_month',
 'fuel_type',
 'brand',
 'unrepaired_damage',
 'ad_created',
 'nr_of_pictures',
 'postal_code',
 'last_seen']

### Missing Values and Outliers

In [176]:
# Calculate percentage of missing features
autos.isnull().sum()/autos.shape[0] * 100

date_crawled           0.000
name                   0.000
seller                 0.000
offer_type             0.000
price                  0.000
abtest                 0.000
vehicle_type          10.190
registration_year      0.000
gearbox                5.360
powerps                0.000
model                  5.516
odometer               0.000
registration_month     0.000
fuel_type              8.964
brand                  0.000
unrepaired_damage     19.658
ad_created             0.000
nr_of_pictures         0.000
postal_code            0.000
last_seen              0.000
dtype: float64

Our dataset is fairly complete, missing no more than 20% of values in any feature.

Only 5 features have missing entries:
* vehicle type (10.1% missing)
* gearbox (5.3% missing)
* model (5.5% missing)
* fuel type (9.0% missing)
* whether damage was repaired (19.7%)

### Data Integrity
Next, we use basic descriptive statistics to check data integrity.

In [179]:
# Descriptive statistics for numeric data
autos.describe()

,registration_year,powerps,registration_month,nr_of_pictures,postal_code
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


In [180]:
# Descriptive statistics for string data
autos.describe(include=['O'])

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,gearbox,model,odometer,fuel_type,brand,unrepaired_damage,ad_created,last_seen
count,50000,50000,50000,50000,50000,50000,44905,47320,47242,50000,45518,50000,40171,50000,50000
unique,48213,38754,2,2,2357,2,8,2,245,13,7,40,2,76,39481
top,2016-03-08 10:40:35,Ford_Fiesta,privat,Angebot,$0,test,limousine,manuell,golf,"150,000km",benzin,volkswagen,nein,2016-04-03 00:00:00,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,36993,4024,32424,30107,10687,35232,1946,8


Observations:
* `nr_of_pictures`, `seller`, and `offer_type` are monovalued, and so contribute no information
* `registration_year` has a maximum of `9999`, which is likely a code for missingness
* `registration_month` has a minimum of `0`, which is likely a code for missingness
* `powerps` measures PS, where 1 PS $\approx$ 0.986 HP. A max PS of `17700` likely signals bad data entries
* `price` has \$0 as its mode; it's unlikely that cars were given away for free, so a $0 perhaps means an expired posting

We'll investigate registration dates, the listed PS, and the price further.

In [181]:
# Highest plausible date is 2019
autos["registration_year"].sort_values(ascending=False).head(20)

38076    9999
33950    9999
14341    9999
8012     9999
6308     9996
13559    9000
49910    9000
25003    8888
8360     6200
27618    5911
24519    5000
49153    5000
4164     5000
22799    5000
42079    4800
453      4500
4549     4100
27578    2800
49185    2019
5763     2019
Name: registration_year, dtype: int64

In [182]:
# Cars were invented in 1885, so lowest plausible date is 1910.
autos["registration_year"].sort_values().head(20)

22316    1000
49283    1001
24511    1111
35238    1500
10556    1800
32585    1800
28693    1910
42181    1910
15898    1910
3679     1910
30781    1910
33295    1910
45157    1910
22659    1910
46213    1910
21416    1927
22101    1929
11246    1931
2573     1934
2221     1934
Name: registration_year, dtype: int64

In [183]:
# 5075 records with registration month 0
autos[autos["registration_month"] == 0].shape[0]

5075

In [223]:
# 54 rows with 1000 or more PS, which is greatly inflated.
# If a model is to be run using PS, these values should probably be 
# manually inspected, or thrown out with little data loss 
autos[autos["powerps"] >= 1000].shape[0]

54

Further observations:
* There are a number of rows with registration years way above `2019`, the year this script was created; these years should be coded as missing
* `5075` rows have `0` as the registration month; a possible imputation could use `1` for January and instead rely on the year if registration date is important
* `54` rows have `1000` or more PS for cars that obviously don't have that PS; this means that PS is fairly clean, but any modeling effort will need to drop or otherwise correct these values

To continue, the `price` and `odometer` columns should be converted to numeric types.

In [185]:
# Check for non-numeric strings
autos["price"].str.replace('[0-9|,]+','').unique() # all rows begin with $
autos["odometer"].str.replace('[0-9|,]+','').unique() # all rows end in km

# Convert price and odometer to float types
autos["price"] = autos["price"].str.replace('[^0-9]','').astype(float)
autos["odometer"] = autos["odometer"].str.replace('[^0-9]','').astype(float)

# Rename odometer to odometer_km to include units in name
autos = autos.rename({"odometer" : "odometer_km"}, axis=1)

In [186]:
# Descriptive statistics of price and odometer_km, formatted to 2 decimals
autos[["price", "odometer_km"]].describe().applymap(lambda x: format(x, '.2f'))

,price,odometer_km
count,50000.00,50000.00
mean,9840.04,125732.70
std,481104.38,40042.21
min,0.00,5000.00
25%,1100.00,125000.00
50%,2950.00,150000.00
75%,7200.00,150000.00
max,99999999.00,150000.00


In [187]:
# Except for the Ferrari F40 (id: 7814) and Ferrari FXX (id: 47634),
# all other prices look obviously typed in to mean missing
autos.sort_values("price", ascending=False).head(20)

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,powerps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
39705,2016-03-22 14:58:27,Tausch_gegen_gleichwertiges,privat,Angebot,99999999.0,control,limousine,1999,automatik,224,s_klasse,150000.0,9,benzin,mercedes_benz,NaN,2016-03-22 00:00:00,0,73525,2016-04-06 05:15:30
42221,2016-03-08 20:39:05,Leasinguebernahme,privat,Angebot,27322222.0,control,limousine,2014,manuell,163,c4,40000.0,2,diesel,citroen,NaN,2016-03-08 00:00:00,0,76532,2016-03-08 20:39:05
27371,2016-03-09 15:45:47,Fiat_Punto,privat,Angebot,12345678.0,control,NaN,2017,NaN,95,punto,150000.0,0,NaN,fiat,NaN,2016-03-09 00:00:00,0,96110,2016-03-09 15:45:47
39377,2016-03-08 23:53:51,Tausche_volvo_v40_gegen_van,privat,Angebot,12345678.0,control,NaN,2018,manuell,95,v40,150000.0,6,NaN,volvo,nein,2016-03-08 00:00:00,0,14542,2016-04-06 23:17:31
47598,2016-03-31 18:56:54,Opel_Vectra_B_1_6i_16V_Facelift_Tuning_Showcar...,privat,Angebot,12345678.0,control,limousine,2001,manuell,101,vectra,150000.0,3,benzin,opel,nein,2016-03-31 00:00:00,0,4356,2016-03-31 18:56:54
2897,2016-03-12 21:50:57,Escort_MK_1_Hundeknochen_zum_umbauen_auf_RS_2000,privat,Angebot,11111111.0,test,limousine,1973,manuell,48,escort,50000.0,3,benzin,ford,nein,2016-03-12 00:00:00,0,94469,2016-03-12 22:45:27
24384,2016-03-21 13:57:51,Schlachte_Golf_3_gt_tdi,privat,Angebot,11111111.0,test,NaN,1995,NaN,0,NaN,150000.0,0,NaN,volkswagen,NaN,2016-03-21 00:00:00,0,18519,2016-03-21 14:40:18
11137,2016-03-29 23:52:57,suche_maserati_3200_gt_Zustand_unwichtig_laufe...,privat,Angebot,10000000.0,control,coupe,1960,manuell,368,NaN,100000.0,1,benzin,sonstige_autos,nein,2016-03-29 00:00:00,0,73033,2016-04-06 21:18:11
47634,2016-04-04 21:25:21,Ferrari_FXX,privat,Angebot,3890000.0,test,coupe,2006,NaN,799,NaN,5000.0,7,NaN,sonstige_autos,nein,2016-04-04 00:00:00,0,60313,2016-04-05 12:07:37
7814,2016-04-04 11:53:31,Ferrari_F40,privat,Angebot,1300000.0,control,coupe,1992,NaN,0,NaN,50000.0,12,NaN,sonstige_autos,nein,2016-04-04 00:00:00,0,60598,2016-04-05 11:34:11


In [188]:
autos[autos["price"] == 0].shape[0] # 1421 rows with 0 price

1421

More observations:
* `1,421` rows have `0` price, which probably means the listing ended
* `9` rows have an absurd price like `12345678`, which likely refer to missing values
* `odometer` appears to be a reliable and clean column

### Dealing with outliers and odd values

#### Prices
Let's set any prices at or above 999,999 to Null, except for the Ferrari entries, since the cars with these values in the dataset are associated with common consumer vehicles and not exotic cars.

In [189]:
# Set to missing any non-Ferrari price greater or equal to 999999
autos.loc[(autos["price"] >= 999999) & ~(autos["name"].str.contains("Ferrari")), "price"] = np.nan

#### Registration Month and Year
Let's set any registration months with a value of 0 to 1 to impute January as the earliest date possibly registered within the year. In addition, we'll truncate any dates before 1910 and after 2019.

This will impute 5075 registration months, and truncate 24 bad registration years.

In [190]:
# Impute missing registration months, and set years above 2019 to NaN
autos.loc[autos["registration_month"] == 0, "registration_month"] = 1
autos.loc[(autos["registration_year"] > 2019) | 
          (autos["registration_year"] < 1910), "registration_year"] = np.nan

### Processing Date Columns
Unlike `registration_month` and `registration_year`, `date_crawled`, `last_seen`, and `ad_created` are datestamps that are not automatically converted by `pandas`, so we'll need to manually convert these to YYYY-MM-DD to do further analysis. 

In [191]:
def describe_date(datestamp_col):
    return(
        (datestamp_col
            .str[:10] # First 10 characters is YYYY-MM-DD
            .value_counts(normalize=True, dropna=False) # summarize distribution
            .sort_index()
        ) # sort by earliest to latest date
    )

In [192]:
describe_date(autos["date_crawled"])

2016-03-05    0.02538
2016-03-06    0.01394
2016-03-07    0.03596
2016-03-08    0.03330
2016-03-09    0.03322
2016-03-10    0.03212
2016-03-11    0.03248
2016-03-12    0.03678
2016-03-13    0.01556
2016-03-14    0.03662
2016-03-15    0.03398
2016-03-16    0.02950
2016-03-17    0.03152
2016-03-18    0.01306
2016-03-19    0.03490
2016-03-20    0.03782
2016-03-21    0.03752
2016-03-22    0.03294
2016-03-23    0.03238
2016-03-24    0.02910
2016-03-25    0.03174
2016-03-26    0.03248
2016-03-27    0.03104
2016-03-28    0.03484
2016-03-29    0.03418
2016-03-30    0.03362
2016-03-31    0.03192
2016-04-01    0.03380
2016-04-02    0.03540
2016-04-03    0.03868
2016-04-04    0.03652
2016-04-05    0.01310
2016-04-06    0.00318
2016-04-07    0.00142
Name: date_crawled, dtype: float64

In [193]:
describe_date(autos["ad_created"])

2015-06-11    0.00002
2015-08-10    0.00002
2015-09-09    0.00002
2015-11-10    0.00002
2015-12-05    0.00002
2015-12-30    0.00002
2016-01-03    0.00002
2016-01-07    0.00002
2016-01-10    0.00004
2016-01-13    0.00002
2016-01-14    0.00002
2016-01-16    0.00002
2016-01-22    0.00002
2016-01-27    0.00006
2016-01-29    0.00002
2016-02-01    0.00002
2016-02-02    0.00004
2016-02-05    0.00004
2016-02-07    0.00002
2016-02-08    0.00002
2016-02-09    0.00004
2016-02-11    0.00002
2016-02-12    0.00006
2016-02-14    0.00004
2016-02-16    0.00002
2016-02-17    0.00002
2016-02-18    0.00004
2016-02-19    0.00006
2016-02-20    0.00004
2016-02-21    0.00006
               ...   
2016-03-09    0.03324
2016-03-10    0.03186
2016-03-11    0.03278
2016-03-12    0.03662
2016-03-13    0.01692
2016-03-14    0.03522
2016-03-15    0.03374
2016-03-16    0.03000
2016-03-17    0.03120
2016-03-18    0.01372
2016-03-19    0.03384
2016-03-20    0.03786
2016-03-21    0.03772
2016-03-22    0.03280
2016-03-23

In [194]:
describe_date(autos["last_seen"])

2016-03-05    0.00108
2016-03-06    0.00442
2016-03-07    0.00536
2016-03-08    0.00760
2016-03-09    0.00986
2016-03-10    0.01076
2016-03-11    0.01252
2016-03-12    0.02382
2016-03-13    0.00898
2016-03-14    0.01280
2016-03-15    0.01588
2016-03-16    0.01644
2016-03-17    0.02792
2016-03-18    0.00742
2016-03-19    0.01574
2016-03-20    0.02070
2016-03-21    0.02074
2016-03-22    0.02158
2016-03-23    0.01858
2016-03-24    0.01956
2016-03-25    0.01920
2016-03-26    0.01696
2016-03-27    0.01602
2016-03-28    0.02086
2016-03-29    0.02234
2016-03-30    0.02484
2016-03-31    0.02384
2016-04-01    0.02310
2016-04-02    0.02490
2016-04-03    0.02536
2016-04-04    0.02462
2016-04-05    0.12428
2016-04-06    0.22100
2016-04-07    0.13092
Name: last_seen, dtype: float64

In [195]:
autos["registration_year"].describe()

count    49976.000000
mean      2003.366836
std          7.693943
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2019.000000
Name: registration_year, dtype: float64

Observations:
* For both `date_crawled` and `ad_created`, it appears that most listings appeared between March 7th and April 4th with about 3% of listings appearing every day between these dates almost uniformly
* Some ads are quite old, going all the way back to 2015
* Cars are more frequently last seen closer to the last date available, April 7th; in addition, almost half of all cars were last seen between April 5th and April 7th
    * This last point is probably because the car was last seen at the same time the ad was crawled
* Registration years ranged from 1910 to 2019, and it appears that the majority of registrations occured between 1999 and 2008, going by IQR

## Data Aggregation

### Average sales for the top 10 car brands

Let's see what the mean prices are for the top 10 most frequent car brands. Noting earlier that there are a number of \$0 prices that are probably implausible, we'll summarize the values by sales with nominal values above \$0.

In [196]:
# Find the mean price for the top 10 brands
top_10_brands = autos["brand"].value_counts().iloc[:10].index
mean_price_top_10 = {}

for b in top_10_brands:
    mean_price_top_10[b] = autos.loc[(autos["brand"] == b) &
                                     (autos["price"] != 0), "price"].mean()

In [199]:
# Sort the dictionary values using itemgetter
from operator import itemgetter
sorted(mean_price_top_10.items(), key=itemgetter(1), reverse=True)

[('audi', 9212.9306621881),
 ('mercedes_benz', 8536.027085124677),
 ('bmw', 8261.382442169132),
 ('volkswagen', 5428.701460771984),
 ('seat', 4315.744565217391),
 ('ford', 3728.4121821407452),
 ('peugeot', 3065.611888111888),
 ('opel', 2944.6075421641085),
 ('fiat', 2793.8700475435817),
 ('renault', 2431.195698924731)]

Executive summary:
* The top 10 brands were Audi, BMW, Fiat, Ford, Mercedes-Benz, Opel, Peugeot, Renault, SEAT, and Volkswagen
* Audi had the highest used car price at \$9,212
* Renault had the lowest used car price at \$2,431
* Audi, Mercedes-Benz, and BMW had the three highest average sale prices
* Opel, Fiat, and Renault had the three lowest average sale prices

### Average mileage of top 6 brands sold

It would be interesting not just to see the top 10 mean prices, but also what kind of average mileage those cars sold had. To facilitate this, we'll present the mean price and mean milage together.

In [200]:
mean_miles_top_10 = {}

for b in top_10_brands:
    mean_miles_top_10[b] = autos.loc[(autos["brand"] == b) &
                                     (autos["price"] != 0), "odometer_km"].mean()


In [201]:
# pandas interprets keys of a list of dictionaries as columns, not as indexes
# so to properly code as columns in a line, we use the index argument and
# transpose the resulting matrix
(pd.DataFrame([pd.Series(mean_price_top_10), pd.Series(mean_miles_top_10)],
             index = ['mean_price', 'mean_mileage'])
   .T
   .applymap(lambda x: format(x, '.0f'))
)

,mean_price,mean_mileage
audi,9213,129493
bmw,8261,132686
fiat,2794,117593
ford,3728,124328
mercedes_benz,8536,130801
opel,2945,129387
peugeot,3066,127357
renault,2431,128062
seat,4316,121641
volkswagen,5429,128903


Executive summary:
* All the top 10 car brands had mean mileages well above between 115,000 - 135,000
* These brands sell even when mileage is very high, indicating some kind of consumer confidence

## Extra Data Cleaning

### Translating German words to English
We can use `Series.replace()` to efficiently translate German words to English words for the categorical variables in the dataset.

The workflow we'll use is to list the unique categories, and then replace any German words with English translations.

In [ ]:
# Strings are stored as 'object' types in pandas
autos.describe(include='O')

`vehicle_type`, `gearbox`, `fuel_type`, and `unrepaired_damage` are informative columns that are worth translating for analytical convenience in English.

#### Vehicle Type

In [ ]:
autos["vehicle_type"].unique() # Before translation

##### Translation Notes:
* [Kombi](https://en.wikipedia.org/wiki/Volkswagen_Type_2) refers to the Voltzwagen Type 2 vehicles. English translations are informal by nature, so I'll opt to keep this value as is
* [Kleinwagen](https://en.wikipedia.org/wiki/Subcompact_car) translates to the American classification *subcompact*, and I use this translation to keep the category to one word


In [ ]:
autos["vehicle_type"].replace({'kleinwagen' : 'subcompact',
                               'cabrio' : 'convertible',
                               'andere' : 'other'},
                              inplace = True)

In [ ]:
autos["vehicle_type"].unique() # After translation

#### Gearbox

In [ ]:
autos["gearbox"].unique() # Before translation

In [ ]:
autos["gearbox"].replace({'manuell' : 'manual',
                          'automatik' : 'automatic'},
                         inplace = True)

In [ ]:
autos["gearbox"].unique() # After translation

#### Fuel Type

In [ ]:
autos["fuel_type"].unique() # Before translation

##### Translation Notes
* *Benzin* refers to the colloquial *gas* in English for petroleum gas, but to keep it simple, I just call it `gas` here
* *LPG* and *CNG* are liquified petroleum gas and compressed natural gas, so these don't need to be translated

In [ ]:
autos["fuel_type"].replace({'benzin' : 'gas',
                            'elektro' : 'electric',
                            'andere' : 'other'},
                           inplace = True)

In [ ]:
autos["fuel_type"].unique() # After translation

##### Unrepaired Damage

In [ ]:
autos["unrepaired_damage"].unique() # Before translation

In [ ]:
# map() is faster than replace(), and only 2 values to replace so
# I opted to use that
autos["unrepaired_damage"] = autos["unrepaired_damage"].map({'nein' : 'no', 'ja' : 'yes'})

In [ ]:
autos["unrepaired_damage"].unique() # After translation

## Extra Analysis

### Most common brand and model combinations
One interesting question to answer is: which models are the most common for each brand?

We'll answer this in a table of all brands and the most popular model in each brand, along with the count of listings in the dataset.

In [ ]:
# Idea: Find the largest count within each brand using grouping
max_brand_model = (autos
                     .groupby(["brand","model"])
                     .size()
                     .groupby(level=0, group_keys=False)
                     .nlargest(n=1)
                     .to_frame('size')
                     .rename({'size' : 'N'}, axis=1)
                  )

In [ ]:
max_brand_model.style

We can also find out which three combinations of brand and model have the most listings.

In [ ]:
max_brand_model.sort_values("N").tail(3)

Looks like the top three most common brand and model listings are:
* Opel Corsa
* BMW 3 Series
* Volkswagen Golf

Together, these three specific brand and models make up 8,520 of the listings, or 17% of the listings.

### Average prices by mileage
It appeared in our earlier analysis of the mileage of the top 10 brands by selling price that many cars with high mileage (115,000+) were being sold. It is possible that 1) high mileage results in lower sale value due to depreciation, and 2) the high mileage cars drag the overall average sale price down.

We can explore this by looking at sales data by mileage to see if there is an overall effect.

In [162]:
(1735+2761+4024) / 50000

0.1704

In [213]:
# Distribution of mileage
autos["odometer_km"].value_counts()

150000.0    32424
125000.0     5170
100000.0     2169
90000.0      1757
80000.0      1436
70000.0      1230
60000.0      1164
50000.0      1027
5000.0        967
40000.0       819
30000.0       789
20000.0       784
10000.0       264
Name: odometer_km, dtype: int64

As suspected, the dataset is mostly represented by cars with 150,000 km or more of mileage. Because of this, quartiles have little meaning here. To see this, let's get simple summary statistics.

In [214]:
autos["odometer_km"].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

As we can see from the summary statistics, the 25th percentile is already 125,000 km, and the median at 150,000 km, or the max of the dataset. Since there aren't too many mileage categories, let's find the average prices for each mileage category.

In [219]:
# Among all nonzero prices, find the mean price for each mileage label
(autos[autos["price"] != 0]
    .groupby("odometer_km")["price"]
    .mean()
    .round()
)

odometer_km
5000.0      13168.0
10000.0     20248.0
20000.0     18054.0
30000.0     16532.0
40000.0     15481.0
50000.0     14982.0
60000.0     12243.0
70000.0     10868.0
80000.0      9614.0
90000.0      8369.0
100000.0     8017.0
125000.0     6139.0
150000.0     3760.0
Name: price, dtype: float64

Aha! Looks like mileage is inversely proportional to average price, meaning that depreciation in the dataset is very real. Cars with higher mileage tend to have lower sale prices.

Interestingly, cars with less than 100,000 km sell on average for at least \$8,000, and prices for low mileage cars of 20,000 km or less sell for at least \$18,000 on average.

### Average price of damaged cars vs non-damaged cars
Similar to our analysis above, we can look at how much damaged cars sell for compared to non-damaged cars.

In [222]:
(autos[autos["price"] != 0]
     .groupby("unrepaired_damage")["price"]
     .mean()
     .round()
)

unrepaired_damage
ja      2435.0
nein    7236.0
Name: price, dtype: float64

For this dataset, non-damaged cars (i.e. cars without unrepaired damage) sell for almost \$5,000 more compared to their damaged counterparts.